In [113]:
import requests
from bs4 import BeautifulSoup
import re

In [22]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}


In [24]:
r = requests.get(f"https://www.amazon.com/s?i=computers&rh=n%3A565108&page=1", headers=headers)
r.content

b'<!--\n        To discuss automated access to Amazon data please contact api-services-support@amazon.com.\n        For information about migrating to our APIs refer to our Marketplace APIs at https://developer.amazonservices.com/ref=rm_5_sv, or our Product Advertising API at https://affiliate-program.amazon.com/gp/advertising/api/detail/main.html/ref=rm_5_ac for advertising use cases.\n-->\n<!doctype html>\n<html>\n<head>\n  <meta charset="utf-8">\n  <meta http-equiv="x-ua-compatible" content="ie=edge">\n  <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n  <title>Sorry! Something went wrong!</title>\n  <style>\n  html, body {\n    padding: 0;\n    margin: 0\n  }\n\n  img {\n    border: 0\n  }\n\n  #a {\n    background: #232f3e;\n    padding: 11px 11px 11px 192px\n  }\n\n  #b {\n    position: absolute;\n    left: 22px;\n    top: 12px\n  }\n\n  #c {\n    position: relative;\n    max-width: 800px;\n    padding: 0 40px 0 0\n  }\n\n  #e, #f {\n    hei

In [42]:
def amazon_laptop_page(page_num):
    r = requests.get(f"https://www.amazon.com/s?i=computers&rh=n%3A565108&fs=true&page=2", headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    #print(soup.content)
    links = soup.html.body.select(".a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal")[3:]
    #print(links)
    names_a = links[::2]
    prices_a = links[1::2]
    hrefs = [n['href'] for n in names_a]
    names = [n.text for n in names_a]
    prices = [p.text for p in prices_a]
    return (hrefs, names, prices)

In [13]:
amazon_laptop_page(2)

None
[]


([], [], [])

In [50]:
def get_website_links(website_func, num_pages, file_name):
    with open(file_name, "w") as amazon_laptop_file:
        for page_num in range(num_pages):
            hrefs, names, prices = website_func(page_num)
            print(f"{len(hrefs)}, {len(names)}, {len(prices)}")
            for tup in zip(hrefs, names, prices):
                print(tup)
                amazon_laptop_file.write("\t".join(tup)+"\n")

In [ ]:
get_website_links(amazon_laptop_page, 10, "amazon_laptop_links.tsv")

## Newegg Requests

In [129]:
def newegg_laptop_page(page_num):
    price_pattern = r"[\d,]+\.\d{2}"
    page_link = f"https://www.newegg.com/Laptops-Notebooks/SubCategory/ID-32/Page-{page_num}"
    r = requests.get(page_link, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    soup = soup.html.body
    items = soup.select(".item-container")
    item_list = []
    for item in items:
        name_html = item.select(".item-title")[0]
        price_html = item.select(".price-current")
        price_match = re.findall(price_pattern, price_html[0].text)
        price = price_match[0] if price_match else None
        #print(f"{name_html},{price_html}")
        item_list.append((
            name_html['href'],
            name_html.text,
            price if price else ""
        ))
    return item_list

def get_newegg_links(num_pages, file_name):
    with open(file_name, "w") as laptop_file:
        for page_num in range(num_pages):
            print(f"page {page_num}")
            for tup in newegg_laptop_page(page_num):
                #print(tup)
                laptop_file.write("\t".join(tup)+"\n")

In [130]:
get_newegg_links(100, "newegg_laptop_links.tsv")

page 0
page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17
page 18
page 19
page 20
page 21
page 22
page 23
page 24
page 25
page 26
page 27
page 28
page 29
page 30
page 31
page 32
page 33
page 34
page 35
page 36
page 37
page 38
page 39
page 40
page 41
page 42
page 43
page 44
page 45
page 46
page 47
page 48
page 49
page 50
page 51
page 52
page 53
page 54
page 55
page 56
page 57
page 58
page 59
page 60
page 61
page 62
page 63
page 64
page 65
page 66
page 67
page 68
page 69
page 70
page 71
page 72
page 73
page 74
page 75
page 76
page 77
page 78
page 79
page 80
page 81
page 82
page 83
page 84
page 85
page 86
page 87
page 88
page 89
page 90
page 91
page 92
page 93
page 94
page 95
page 96
page 97
page 98
page 99


# Getting Data From Product Pages

In [133]:
import pandas as pd

In [138]:
newegg_df = pd.read_csv("newegg_laptop_links.tsv", encoding = 'unicode_escape', sep="\t", header=None)

In [139]:
newegg_df

,0,1,2
0,https://www.newegg.com/pine-gray-asus-zenbook-...,"ASUS ZenBook 13 Ultra-Slim Laptop, 13.3"" OLED ...",849.99
1,https://www.newegg.com/black-lenovo-20uw0012us...,"Lenovo ThinkPad X12 Detachable Gen 1 Laptop, 1...",949.00
2,https://www.newegg.com/acer-sf514-55ta-77ww/p/...,Acer Intel EVO Laptop Swift 5 SF514-55TA-77WW ...,999.99
3,https://www.newegg.com/intel-nuc-m15-lapbc710/...,"Helix WorkPlex 1170EN Premium 15.6"" Touchscree...","1,249.99"
4,https://www.newegg.com/indie-black-asus-vivobo...,"ASUS VivoBook 15 OLED K513 Laptop, 15.6 OLED D...",919.99
...,...,...,...
3756,https://www.newegg.com/quiet-blue-asus-vivoboo...,ASUS Laptop VivoBook Pro K3500PC-DH59-CA Intel...,"1,098.22"
3757,https://www.newegg.com/p/1TS-000A-0CCK4,"Dell Inspiron 14 5000 Laptop Computer 14"" FHD ...",549.00
3758,https://www.newegg.com/p/1TS-000D-09G70,"HP Laptop 15.6"" Screen, 10th Gen Intel Core i7...",799.00
3759,https://www.newegg.com/p/2SN-0001-01B62,"Apple MacBook Air with Apple M1 Chip (13-inch,...","1,069.95"
